<a href="https://colab.research.google.com/github/VasaviLagishetty/COVID19/blob/master/COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests
from datetime import datetime

In [2]:
raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
df = pd.DataFrame(raw_json["features"])
df

,attributes,geometry
0,"{'OBJECTID': 1, 'Province_State': 'Alberta', '...","{'x': -116.57649999999995, 'y': 53.93330000000..."
1,"{'OBJECTID': 2, 'Province_State': 'Anguilla', ...","{'x': -63.06859999999995, 'y': 18.220600000000..."
2,"{'OBJECTID': 3, 'Province_State': 'Anhui', 'Co...","{'x': 117.2264, 'y': 31.82570000000004}"
3,"{'OBJECTID': 4, 'Province_State': 'Aruba', 'Co...","{'x': -69.96829999999994, 'y': 12.521100000000..."
4,"{'OBJECTID': 5, 'Province_State': 'Australian ...","{'x': 149.01240000000007, 'y': -35.47349999999..."
...,...,...
320,"{'OBJECTID': 321, 'Province_State': 'Ohio', 'C...","{'x': -82.76491499999997, 'y': 40.388783000000..."
321,"{'OBJECTID': 322, 'Province_State': 'Alabama',...","{'x': -86.90229999999997, 'y': 32.31820000000005}"
322,"{'OBJECTID': 323, 'Province_State': 'North Car...","{'x': -79.80641899999995, 'y': 35.630066000000..."
323,"{'OBJECTID': 324, 'Province_State': 'District ...","{'x': -77.02681699999994, 'y': 38.89743800000008}"


In [3]:
df["attributes"][0]

{'Confirmed': 5766,
 'Country_Region': 'Canada',
 'Deaths': 95,
 'Last_Update': 1588599159000,
 'Lat': 53.9333,
 'Long_': -116.5765,
 'OBJECTID': 1,
 'Province_State': 'Alberta',
 'Recovered': 0}

In [4]:
df['geometry'][0]

{'x': -116.57649999999995, 'y': 53.93330000000003}

In [5]:
data_list = df["attributes"].tolist()
data = pd.DataFrame(data_list)
data = data.set_index("OBJECTID")
data = data.reset_index(drop = True)
data

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,Alberta,Canada,1.588599e+12,53.933300,-116.576500,5766,0,95
1,Anguilla,United Kingdom,1.588599e+12,18.220600,-63.068600,3,3,0
2,Anhui,China,1.588599e+12,31.825700,117.226400,991,985,6
3,Aruba,Netherlands,1.588599e+12,12.521100,-69.968300,100,81,2
4,Australian Capital Territory,Australia,1.588599e+12,-35.473500,149.012400,107,103,3
...,...,...,...,...,...,...,...,...
320,Ohio,US,1.588599e+12,40.388783,-82.764915,19914,0,1039
321,Alabama,US,1.588599e+12,32.318200,-86.902300,7888,0,290
322,North Carolina,US,1.588599e+12,35.630066,-79.806419,11773,0,435
323,District of Columbia,US,1.588599e+12,38.897438,-77.026817,5016,0,251


In [6]:
bool_series = pd.isnull(data['Last_Update']) 
data[bool_series]

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
279,American Samoa,US,NaN,-14.271,-170.132,0,0,0
290,United States Virgin Islands,US,NaN,18.350,-64.933,0,0,0


In [7]:
data = data[data['Last_Update'].notna()]
data["Province_State"].fillna(value="", inplace=True)

data

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,Alberta,Canada,1.588599e+12,53.933300,-116.576500,5766,0,95
1,Anguilla,United Kingdom,1.588599e+12,18.220600,-63.068600,3,3,0
2,Anhui,China,1.588599e+12,31.825700,117.226400,991,985,6
3,Aruba,Netherlands,1.588599e+12,12.521100,-69.968300,100,81,2
4,Australian Capital Territory,Australia,1.588599e+12,-35.473500,149.012400,107,103,3
...,...,...,...,...,...,...,...,...
320,Ohio,US,1.588599e+12,40.388783,-82.764915,19914,0,1039
321,Alabama,US,1.588599e+12,32.318200,-86.902300,7888,0,290
322,North Carolina,US,1.588599e+12,35.630066,-79.806419,11773,0,435
323,District of Columbia,US,1.588599e+12,38.897438,-77.026817,5016,0,251


In [8]:
def convertTime(t):
  t = int(t)
  return datetime.fromtimestamp(t)

data['Last_Update'] = data['Last_Update']/1000
data['Last_Update'] = data['Last_Update'].apply(convertTime)
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,Alberta,Canada,2020-05-04 13:32:39,53.933300,-116.576500,5766,0,95
1,Anguilla,United Kingdom,2020-05-04 13:32:39,18.220600,-63.068600,3,3,0
2,Anhui,China,2020-05-04 13:32:39,31.825700,117.226400,991,985,6
3,Aruba,Netherlands,2020-05-04 13:32:39,12.521100,-69.968300,100,81,2
4,Australian Capital Territory,Australia,2020-05-04 13:32:39,-35.473500,149.012400,107,103,3
...,...,...,...,...,...,...,...,...
320,Ohio,US,2020-05-04 13:32:39,40.388783,-82.764915,19914,0,1039
321,Alabama,US,2020-05-04 13:32:39,32.318200,-86.902300,7888,0,290
322,North Carolina,US,2020-05-04 13:32:39,35.630066,-79.806419,11773,0,435
323,District of Columbia,US,2020-05-04 13:32:39,38.897438,-77.026817,5016,0,251


In [0]:
df_total = data.groupby('Country_Region',as_index=False).agg({'Confirmed' : 'sum','Recovered' : 'sum','Deaths' : 'sum'}) 

In [10]:
df_total

,Country_Region,Confirmed,Recovered,Deaths
0,Afghanistan,2894,397,90
1,Albania,803,543,31
2,Algeria,4474,1936,463
3,Andorra,748,493,45
4,Angola,35,11,2
...,...,...,...,...
182,West Bank and Gaza,353,77,2
183,Western Sahara,6,5,0
184,Yemen,10,1,2
185,Zambia,137,78,3


In [11]:
total_confirmed = data['Confirmed'].sum()
total_recovered = data['Recovered'].sum()
total_deaths = data['Deaths'].sum()
print("Total confirmed cases till",data['Last_Update'][0],"are",total_confirmed)
print("Total recovered cases till",data['Last_Update'][0],"are",total_recovered)
print("Total death cases till",data['Last_Update'][0],"are",total_deaths)

Total confirmed cases till 2020-05-04 13:32:39 are 3529408
Total recovered cases till 2020-05-04 13:32:39 are 1133538
Total death cases till 2020-05-04 13:32:39 are 248025


In [0]:
df_top10 = df_total.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top10 = df_total.nlargest(10, "Recovered")
top10_countries_2 = df_top10["Country_Region"].tolist()
top10_recovered = df_top10["Recovered"].tolist()

df_top10 = df_total.nlargest(10, "Deaths")
top10_countries_3 = df_top10["Country_Region"].tolist()
top10_deaths = df_top10["Deaths"].tolist()

# Subplots

In [0]:
fig = make_subplots(
    rows = 4, cols = 6,
    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)

In [14]:
message = data["Country_Region"] + " " + data["Province_State"] + "<br>"
message += "Confirmed: " + data["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + data["Deaths"].astype(str) + "<br>"
message += "Recovered: " + data["Recovered"].astype(str) + "<br>"
message += "Last updated: " + data["Last_Update"].astype(str)
data["text"] = message
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,text
0,Alberta,Canada,2020-05-04 13:32:39,53.933300,-116.576500,5766,0,95,Canada Alberta<br>Confirmed: 5766<br>Deaths: 9...
1,Anguilla,United Kingdom,2020-05-04 13:32:39,18.220600,-63.068600,3,3,0,United Kingdom Anguilla<br>Confirmed: 3<br>Dea...
2,Anhui,China,2020-05-04 13:32:39,31.825700,117.226400,991,985,6,China Anhui<br>Confirmed: 991<br>Deaths: 6<br>...
3,Aruba,Netherlands,2020-05-04 13:32:39,12.521100,-69.968300,100,81,2,Netherlands Aruba<br>Confirmed: 100<br>Deaths:...
4,Australian Capital Territory,Australia,2020-05-04 13:32:39,-35.473500,149.012400,107,103,3,Australia Australian Capital Territory<br>Conf...
...,...,...,...,...,...,...,...,...,...
320,Ohio,US,2020-05-04 13:32:39,40.388783,-82.764915,19914,0,1039,US Ohio<br>Confirmed: 19914<br>Deaths: 1039<br...
321,Alabama,US,2020-05-04 13:32:39,32.318200,-86.902300,7888,0,290,US Alabama<br>Confirmed: 7888<br>Deaths: 290<b...
322,North Carolina,US,2020-05-04 13:32:39,35.630066,-79.806419,11773,0,435,US North Carolina<br>Confirmed: 11773<br>Death...
323,District of Columbia,US,2020-05-04 13:32:39,38.897438,-77.026817,5016,0,251,US District of Columbia<br>Confirmed: 5016<br>...


In [15]:
fig.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = data["Long_"],
        lat = data["Lat"],
        hovertext = data["text"],
        showlegend=False,
        marker = dict(
            size = 5,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(50, 50, 50)'
            ),
            cmin = 0,
            color = data['Confirmed'],
            cmax = data['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),    
    row=1, col=1
)

fig.update_geos(
    projection_type="natural earth",
    landcolor="grey",
    oceancolor="LightBlue",
    showocean=True,
    lakecolor="LightBlue"
)



In [16]:
# Confirmed Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=4
)

# Recovered Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=5
)

# Death Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths Cases",
    ),
    row=1, col=6
)

In [17]:
fig.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=2, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="Green"), 
        showlegend=True),
    row=3, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths Cases",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=4, col=4
)

In [28]:
fig.update_layout(
    template="plotly_dark",
    title = "Global COVID-19 Cases (Last Updated: " + str(data["Last_Update"][0]) + ")",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y = -0.1),
    font=dict(family='Candara', size=12, color='#FFFFFF')
)